In [2]:
!pip install transformers datasets torch


In [3]:
!pip install bitsandbytes
!pip install transformers datasets torch


!pip install accelerate
!pip install git+https://github.com/huggingface/transformers.git
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install optimum
!pip install -U accelerate peft

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-rthnhr97
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-rthnhr97
  Resolved https://github.com/huggingface/transformers.git to commit bdb9106f247fca48a71eb384be25dbbd29b065a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9128517 sha256=7a5a431632e45682ae41061e40947bb2fe8307a0c224e74413c5deeccc09818c
  Stored in directory: /tmp/pip-ephem-wheel-cache-p3w5u6rs/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1
ERROR: pip's dependency reso

In [ ]:
import os
import json
import torch
from torchvision import transforms
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the WizardLM summarization model and tokenizer directly onto the selected device
model_name_or_path = "TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ"
model_wizard = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="cuda:0")
tokenizer_wizard = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# Define preprocessing transforms for images
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to preprocess text for WizardLM summarization
def preprocess_text(multi_para_text):
    # Replace newline characters with spaces to create a single line of text
    return multi_para_text.replace('\n', ' ').strip()

# Function to generate summary using WizardLM model
def generate_summary(title, description, llava_generated_text, transcript_text):
    # Combine all text inputs for summarization
    input_text = f"Title:{title}\nDescription: {description}\nLLava-generated Text: {llava_generated_text}\nTranscript Text: {transcript_text}"
    # Preprocess the input text to handle multiple paragraphs
    processed_text = preprocess_text(input_text)

    prompt = f'''Create a detailed, concise, and coherent summary of the video advertisement. Describe the sequence of events or actions in the video, incorporating key information from the title, description, LLava-generated text, and transcript text. Ensure the summary is grammatically correct, concise, and focuses on the most important aspects. Summary should be of 25 words, not exceeding 30 words :
{processed_text}'''

    prompt_template = f'''You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

    pipe = pipeline(
        "text-generation",
        model=model_wizard,
        tokenizer=tokenizer_wizard,
        max_new_tokens=4000,  # Adjust based on model capability
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        eos_token_id=tokenizer_wizard.eos_token_id
    )

    response = pipe(prompt_template)[0]['generated_text']
    generated_summary = response[len(prompt_template):].strip()

    # Post-process to limit to max_words, attempting to preserve meaning
    max_words = 30  # Desired maximum number of words in the summary
    words = generated_summary.split()
    if len(words) > max_words:
        truncated_summary = ' '.join(words[:max_words])
        last_period = truncated_summary.rfind('.')
        if last_period != -1 and last_period + 1 < len(truncated_summary):
            truncated_summary = truncated_summary[:last_period + 1]
    else:
        truncated_summary = generated_summary

    return truncated_summary

# Main loop to process dataset folders
data_path = "/content/"

# Load transcripts
with open(os.path.join(data_path, "transcripts_mod.json"), "r") as f:
    transcripts = json.load(f)

# Load LLava-generated text
with open(os.path.join(data_path, "llava_output.json"), "r") as f:
    llava_output = json.load(f)

# Load metadata
with open(os.path.join(data_path, "metadata.json"), "r") as f:
    metadata = json.load(f)

results = {}

for video_number, transcript_text in transcripts.items():
    # Extract title and description from metadata
    title = metadata.get(video_number, {}).get("title", "")
    description = metadata.get(video_number, {}).get("description", "")
    llava_generated_text = llava_output.get(video_number, "")

    # Generate summary
    summary = generate_summary(title, description, llava_generated_text, transcript_text)
    results[video_number] = summary

# Write the results to a JSON file
output_file = "/content/summary_output.json"
with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)

print("Summary output saved to:", output_file)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
